In [66]:
#imports
from fastbook import *
from fastai.vision.widgets import *
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from shutil import copyfile

In [32]:
#get file names
path = Path('images')
fns = get_image_files(path)
fns

(#1659) [Path('images/chocolate chip/00000000.jpg'),Path('images/chocolate chip/00000001.jpg'),Path('images/chocolate chip/00000002.jpg'),Path('images/chocolate chip/00000003.jpg'),Path('images/chocolate chip/00000004.jpg'),Path('images/chocolate chip/00000005.jpg'),Path('images/chocolate chip/00000006.jpg'),Path('images/chocolate chip/00000007.jpg'),Path('images/chocolate chip/00000008.jpg'),Path('images/chocolate chip/00000009.jpg')...]

In [33]:
#verify 
failed = verify_images(fns)
failed

/home/cstorm125/venv/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:793: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))
/home/cstorm125/venv/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:793: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))
/home/cstorm125/venv/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:793: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


(#0) []

In [67]:
#create dataset
df = pd.DataFrame({'fname':fns})
df['ext'] = df.fname.map(lambda x: x.suffix)
df['label'] = df.fname.map(lambda x: x.parts[-2])
df

,fname,ext,label
0,images/chocolate chip/00000000.jpg,.jpg,chocolate chip
1,images/chocolate chip/00000001.jpg,.jpg,chocolate chip
2,images/chocolate chip/00000002.jpg,.jpg,chocolate chip
3,images/chocolate chip/00000003.jpg,.jpg,chocolate chip
4,images/chocolate chip/00000004.jpg,.jpg,chocolate chip
...,...,...,...
1654,images/raisin/00000895.jpg,.jpg,raisin
1655,images/raisin/00000896.jpg,.jpg,raisin
1656,images/raisin/00000897.jpg,.jpg,raisin
1657,images/raisin/00000898.jpg,.jpg,raisin


In [68]:
#get only .jpg
df = df[df.ext=='.jpg']
df.label.value_counts()

chocolate chip    807
raisin            743
Name: label, dtype: int64

In [72]:
#randomize validation flag
np.random.seed(125)
p = 0.2
df['splits'] = np.random.choice(a=['valid', 'train'], size=df.shape[0], p=[p,1-p])
df.splits.value_counts() / df.shape[0]

/home/cstorm125/venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


train    0.800645
valid    0.199355
Name: splits, dtype: float64

In [73]:
df[df.splits=='train'].label.value_counts()

chocolate chip    651
raisin            590
Name: label, dtype: int64

In [74]:
df[df.splits=='valid'].label.value_counts()

chocolate chip    156
raisin            153
Name: label, dtype: int64

In [86]:
#move them to `train` and `valid` folders
#then curate by hand the validation set
#curate some from training set as well

# df.apply(lambda row: copyfile(row['fname'], f'{row.fname.parts[0]}/{row.splits}/{row.fname.parts[1]}/{row.fname.parts[2]}'),1)